# North Carolina Scorecard

This notebook generates the county, school district, and statewide "future voter scorecards" for NC. It is generalize to be updated every month, with minimal changes.

Scorecard outputs (tables) are written back to BigQuery, where they are then read into Google Sheets for formatting

In [151]:
import pandas as pd
import numpy as np
import pandas_gbq

## Inputs
Update the fields below each month

In [152]:
# Inputs
as_of_data_date = pd.Timestamp("2025-06-07")
acs_year = '2022' # 2022 for 2024 scorecards. ACS vintages trail by 2 years


## Outputs
Run the cells below, without edits each month

In [153]:
# Derived variables
latest_18_yob = as_of_data_date.year - 18 # 2006 for 2024 scorecards
earliest_18_yob = latest_18_yob - 1 # 2005 for 2024 scorecards

latest_45_yob = as_of_data_date.year - 45 # 1979 for 2024 scorecards
earliest_45_yob = latest_45_yob - 1 # 1978 for 2024 scorecards

data_date_suffix = str(as_of_data_date.year) + str(as_of_data_date.month).rjust(2, "0") + str(as_of_data_date.day).rjust(2, "0")  # this is the rolling "as of" date, where we snap the line for 18yos

# Define columns

    # Voter file
REG_YOB_LATE_YEAR = 'REG_YOB_' + str(latest_18_yob) # col name for number of registrants with given birth year (2006 for 2024 scorecards)
REG_YOB_EARLY_YEAR = 'REG_YOB_' + str(earliest_18_yob) # col name for number of registrants with given birth year (2005 for 2024 scorecards)

REG_45_PLUS_YOB_LATE_YEAR = 'REG_45_PLUS_YOB_' + str(latest_45_yob) # col name for number of registrants with given birth year (1979 for 2024 scorecards)
REG_45_PLUS_YOB_EARLY_YEAR = 'REG_45_PLUS_YOB_' + str(earliest_45_yob) # col name for number of registrants with given birth year (1978 for 2024 scorecards)

    # ACS
EST_18_YO_THIS_YEAR = 'EST_18_YO_' + str(as_of_data_date.year) # col name for number of 18 yos this "as_of" year estimated from ACS
EST_18_AND_19_YO_THIS_YEAR = 'EST_18_AND_19_YO_' + str(as_of_data_date.year) # col name for number of 18 and 19 yos this "as_of" year estimated from ACS

EST_45_PLUS_YO_THIS_YEAR = 'EST_45_PLUS_YO_' + str(as_of_data_date.year) # col name for number of 45+ yos in the "as of" year estimated from ACS


### County Scorecard

In [154]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
voter_source_table = data_date_suffix + "_nc_voter_registration"

acs_S0101_table = "S0101_us_counties_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file by county, counting the number of registrants in a given birth year.
* Then, left joins the county estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [155]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH addresses AS(
    SELECT
    ncid as VOTER_ID,
    CASE 
        -- WATAUGA 
        WHEN county_id = 95 AND (mail_addr1 LIKE 'ASU %') THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
        -- ORANGE 
        WHEN county_id = 68 AND res_city_desc = 'CHAPEL HILL' THEN FARM_FINGERPRINT(CONCAT(COALESCE(REGEXP_REPLACE(res_street_address, '( #.*)$', '') ,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
   
        ELSE FARM_FINGERPRINT(CONCAT(COALESCE(res_street_address,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
    END as ADDRESS_ID
    FROM `tcc-research.nc_sources.""" + voter_source_table + """`


), young_voters AS(
    
    SELECT  
    a.*,
    ADDRESS_ID 
    FROM `tcc-research.nc_production.""" + voter_file_table + """` a
    LEFT JOIN addresses on addresses.VOTER_ID = a.VOTER_ID
    WHERE VOTER_STATUS IN ('ACTIVE')

),address_count_18 AS(    
    SELECT  
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
    FROM young_voters a
    WHERE YEAR_OF_BIRTH IN (""" + str(latest_18_yob) + ", " + str(earliest_18_yob) + """)
    GROUP BY ADDRESS_ID

),voter_file_county AS(
    SELECT
    COUNTY_FIPS,
    COUNTY_NAME,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = """ + str(latest_18_yob) + ") AS " + REG_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH = """ + str(earliest_18_yob) + ") AS " + REG_YOB_EARLY_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(latest_45_yob) + ") AS " + REG_45_PLUS_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(earliest_45_yob) + ") AS " + REG_45_PLUS_YOB_EARLY_YEAR + """,
    FROM young_voters
    LEFT JOIN address_count_18 on young_voters.ADDRESS_ID = address_count_18.ADDRESS_ID
    WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0)<4
    GROUP BY COUNTY_FIPS, COUNTY_NAME

), acs_county AS(
    SELECT
    COUNTY_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS """ + EST_18_YO_THIS_YEAR + """,
    EST_15_TO_17_YO * 2 / 3 AS """ + EST_18_AND_19_YO_THIS_YEAR + """,
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
    FROM `tcc-research.acs_sources.""" + acs_S0101_table + """`
    WHERE STATE_FIPS = "37"

)

SELECT
voter_file_county.*,
acs_county.EST_15_TO_17_YO,
acs_county.MOE_15_TO_17_YO,
acs_county.""" + EST_18_YO_THIS_YEAR + """,
acs_county.""" + EST_18_AND_19_YO_THIS_YEAR + """,
acs_county.""" + EST_45_PLUS_YO_THIS_YEAR + """

FROM voter_file_county LEFT JOIN acs_county ON voter_file_county.COUNTY_FIPS = acs_county.COUNTY_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [156]:
# Preview
df.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,049,CRAVEN,65646,266,985,42651,41794,3499,138,1166.333333,2332.666667,50153
1,037,CHATHAM,56611,240,732,39605,38860,3022,243,1007.333333,2014.666667,46377
2,099,JACKSON,24781,82,471,14885,14552,1477,162,492.333333,984.666667,20839
3,171,SURRY,41934,200,679,27497,26979,2843,104,947.666667,1895.333333,39691
4,045,CLEVELAND,58957,245,887,37994,37188,3853,115,1284.333333,2568.666667,52391


In [157]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [158]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.5659340659340659


In [159]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [160]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250607,EST_REG_RATE_18_YO_AS_OF_20250607
7,183,WAKE,733680,4085,12607,401072,388259,47558,30,15852.666667,31705.333333,499266,11219.730769,0.70775
74,119,MECKLENBURG,678609,3426,10547,348054,336915,42911,18,14303.666667,28607.333333,465789,9394.906593,0.656818
29,081,GUILFORD,324598,1496,5340,190867,186017,21093,*****,7031.000000,14062.000000,253633,4518.087912,0.642595
71,067,FORSYTH,227605,1127,3591,136195,132974,15559,27,5186.333333,10372.666667,186737,3159.269231,0.609153
68,063,DURHAM,196155,793,2509,100883,97901,10224,39,3408.000000,6816.000000,139455,2212.928571,0.649334


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [161]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.4328767123287671


In [162]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [163]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head(20)

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250607,EST_REG_RATE_18_YO_AS_OF_20250607,EST_REG_45_PLUS_YO_AS_OF_20250607,EST_REG_RATE_45_PLUS_YO_AS_OF_20250607
7,183,WAKE,733680,4085,12607,401072,388259,47558,30,15852.666667,31705.333333,499266,11219.730769,0.70775,393805.449315,0.788769
74,119,MECKLENBURG,678609,3426,10547,348054,336915,42911,18,14303.666667,28607.333333,465789,9394.906593,0.656818,341736.813699,0.733673
29,081,GUILFORD,324598,1496,5340,190867,186017,21093,*****,7031.000000,14062.000000,253633,4518.087912,0.642595,188116.452055,0.741688
71,067,FORSYTH,227605,1127,3591,136195,132974,15559,27,5186.333333,10372.666667,186737,3159.269231,0.609153,134368.29589,0.719559
68,063,DURHAM,196155,793,2509,100883,97901,10224,39,3408.000000,6816.000000,139455,2212.928571,0.649334,99191.838356,0.711282
55,021,BUNCOMBE,180907,730,2096,110982,108033,8663,38,2887.666667,5775.333333,144295,1916.197802,0.66358,109309.553425,0.757542
19,051,CUMBERLAND,172886,875,2979,97887,95206,12860,36,4286.666667,8573.333333,129853,2560.917582,0.597415,96366.542466,0.74212
48,129,NEW HANOVER,157351,597,2114,94523,92269,7279,81,2426.333333,4852.666667,115498,1793.384615,0.739134,93244.70411,0.807327
80,179,UNION,155725,1157,3363,94295,91390,12970,5,4323.333333,8646.666667,115787,3060.236264,0.707842,92647.506849,0.800155
20,101,JOHNSTON,143652,820,2608,83052,80597,9984,98,3328.000000,6656.000000,103011,2295.956044,0.689891,81659.712329,0.792728


In [164]:
df_reg_est = df_reg_est.sort_values('EST_REG_RATE_18_YO_AS_OF_20250607', ascending=False)
df_reg_est

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250607,EST_REG_RATE_18_YO_AS_OF_20250607,EST_REG_45_PLUS_YO_AS_OF_20250607,EST_REG_RATE_45_PLUS_YO_AS_OF_20250607
37,177,TYRRELL,1882,1,21,1362,1335,36,39,12.000000,24.000000,1839,12.884615,1.073718,1346.687671,0.732293
77,003,ALEXANDER,23139,245,343,15079,14789,1341,121,447.000000,894.000000,20246,439.115385,0.982361,14914.534247,0.736666
67,189,WATAUGA,33935,144,410,17686,17264,1202,29,400.666667,801.333333,23535,376.032967,0.938518,17446.673973,0.741308
51,031,CARTERET,50435,247,660,36164,35541,2231,114,743.666667,1487.333333,42921,620.516484,0.834401,35810.682192,0.834339
39,141,PENDER,44416,265,698,28276,27495,2379,130,793.000000,1586.000000,32899,660.021978,0.83231,27833.076712,0.846016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,185,WARREN,11413,30,144,8262,8102,681,70,227.000000,454.000000,11456,111.494505,0.491165,8171.260274,0.713273
17,173,SWAIN,8240,34,108,5501,5363,584,98,194.666667,389.333333,7158,95.120879,0.488635,5422.736986,0.757577
90,103,JONES,6175,17,79,4218,4119,392,71,130.666667,261.333333,5813,61.708791,0.472261,4161.854795,0.715956
70,033,CASWELL,13409,33,153,9233,9071,783,46,261.000000,522.000000,13432,119.587912,0.458191,9141.126027,0.680548


#### Output
Write back to BQ

##### Wide

In [165]:
# Flag largest counties
df_reg_est['is_in_10_largest'] = np.where(df_reg_est.COUNTY_NAME.isin(df_reg_est.nlargest(10, columns='EST_18_YO_2025').COUNTY_NAME),1,0)

In [166]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_county_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 4718.00it/s]


### Statewide Scorecard

In [167]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
acs_S0101_table = "S0101_us_states_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file for NC state, counting the number of registrants in a given birth year.
* Then, left joins the statewide estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos is derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [168]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH addresses AS(
    SELECT
    ncid as VOTER_ID,
    CASE 
        -- WATAUGA 
        WHEN county_id = 95 AND (mail_addr1 LIKE 'ASU %') THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
        -- ORANGE 
        WHEN county_id = 68 AND res_city_desc = 'CHAPEL HILL' THEN FARM_FINGERPRINT(CONCAT(COALESCE(REGEXP_REPLACE(res_street_address, '( #.*)$', '') ,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
   
        ELSE FARM_FINGERPRINT(CONCAT(COALESCE(res_street_address,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
    END as ADDRESS_ID
    FROM `tcc-research.nc_sources.""" + voter_source_table + """`


), young_voters AS(
    
    SELECT  
    a.*,
    ADDRESS_ID 
    FROM `tcc-research.nc_production.""" + voter_file_table + """` a
    LEFT JOIN addresses on addresses.VOTER_ID = a.VOTER_ID
    WHERE VOTER_STATUS IN ('ACTIVE')

),address_count_18 AS(    
    SELECT  
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
    FROM young_voters a
    WHERE YEAR_OF_BIRTH IN (""" + str(latest_18_yob) + ", " + str(earliest_18_yob) + """)
    GROUP BY ADDRESS_ID

),voter_file_nc AS(
    SELECT
    STATE_FIPS,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = """ + str(latest_18_yob) + ") AS " + REG_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH = """ + str(earliest_18_yob) + ") AS " + REG_YOB_EARLY_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(latest_45_yob) + ") AS " + REG_45_PLUS_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(earliest_45_yob) + ") AS " + REG_45_PLUS_YOB_EARLY_YEAR + """,
    FROM young_voters
    LEFT JOIN address_count_18 on young_voters.ADDRESS_ID = address_count_18.ADDRESS_ID
    WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0)<4
    GROUP BY STATE_FIPS

), acs_nc AS(
    SELECT
    STATE_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS """ + EST_18_YO_THIS_YEAR + """,
    EST_15_TO_17_YO * 2 / 3 AS """ + EST_18_AND_19_YO_THIS_YEAR + """,
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
    FROM `tcc-research.acs_sources.""" + acs_S0101_table + """`
    WHERE STATE_FIPS = "37"

)

SELECT
voter_file_nc.*,
acs_nc.EST_15_TO_17_YO,
acs_nc.MOE_15_TO_17_YO,
acs_nc.""" + EST_18_YO_THIS_YEAR + """,
acs_nc.""" + EST_18_AND_19_YO_THIS_YEAR + """,
acs_nc.""" + EST_45_PLUS_YO_THIS_YEAR + """

FROM voter_file_nc LEFT JOIN acs_nc ON voter_file_nc.STATE_FIPS = acs_nc.STATE_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [169]:
# Preview
df

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,37,6486583,31737,102477,3909600,3813094,404849,794,134949.666667,269899.333333,5139416


In [170]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [171]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.5659340659340659


In [172]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [173]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250607,EST_REG_RATE_18_YO_AS_OF_20250607
0,37,6486583,31737,102477,3909600,3813094,404849,794,134949.666667,269899.333333,5139416,89732.225275,0.664931


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [174]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.4328767123287671


In [175]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [176]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250607,EST_REG_RATE_18_YO_AS_OF_20250607,EST_REG_45_PLUS_YO_AS_OF_20250607,EST_REG_RATE_45_PLUS_YO_AS_OF_20250607
0,37,6486583,31737,102477,3909600,3813094,404849,794,134949.666667,269899.333333,5139416,89732.225275,0.664931,3854869.2,0.75006


#### Output
Write back to BQ

In [177]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_statewide_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 4788.02it/s]
